# prepare_Baden2016_data
This notebook loads the dataset of stimulus responses of the total RGC population (recorded using the Calcium-indicator OGB-1 with a two-photon microscope) from Baden et al. (2016, Nature) and extracts data relevant to the RGC cluster assignment of the new data set of dLGN-projecting RGC responses (Calcium-indicator GCaMP6f).

# Instructions
Data to download
- data from Baden2016
  - from https://datadryad.org/resource/doi:10.5061/dryad.d9v38, download "BadenEtAl_RGCs_2016_v1.mat" and put into "../../data/2P/original/"
    - this also contains a readme with extended data info
- (De-)convolution kernels
  - put "ogb1_kern.mat" and "gcamp6f_kern.mat" into "../../data/2P/original/"

# Short Baden2016 data info
- chirp_avg: median of all repeats, scaled to max(abs(x))=1
- use group_idx for grouping avg respose (remaining clusters after SNR-criterion; <> cluster_idx contains all clusters)
 - group_idx: RGCs: 1-28, uncertain RGCs: 29-32, other: 33-46
 - clu_idx: RGCs: 1-39, uncertain RGCs: 40-49, other: 50-76

# Setup

In [1]:
## Library setup

# Misc libs
from __future__ import print_function
import warnings

# Data libs
import sys
import os # directory lib
import h5py # HDF5 lib
import numpy as np # linear alg lib
import scipy.io as sio
from scipy import interpolate
import time

# Custom modules
sys.path.append('../utils/') # Import path to utility functions
import postProc as pop # Import PostProcessing module (functions: normalization, interpolation, QI etc.)

%matplotlib inline 

In [2]:
# Parameter setup
p = {}
p.update({
        # Choose if analysing groups or clusters
        'groupVClu': 'clu', # OPTIONS: 'group', 'clu'
        
        # IO pars
        'loadDir': "../../data/2P/original/", # load directory
        'loadFid': "BadenEtAl_RGCs_2016_v1.mat",
        'saveDir': "../../data/2P/proc/", # save directory        
        'saveFid': 'BadenEtAl_RGCs_2016_ogb1', # file ID for group ana        
        'fileFmt': '.mat', # file format

        ## RGC-cells (see pre-info)
        
        # Set stimuli of interest & var names for dataframe accordingly
        'stim': ["chirp", "bar"], # stimulus name
        
        # Cell type pars
        'rgcRangeGroup': [1, 32], # Set range of RGC cell groups (see info)
        'rgcRangeClu': [1, 49], # Set range of RGC cell clusters (see info)
        
        # Normalization pars
        'doNorm': True, # Set if to do normalization or not
        'normMode': 'medMax', # see normalization function in utils/postProc.py
        
        # Resampling pars
        'doResample': False,
        'newSRate': 40, # Desired sampling rate in Hz (old: 7.825)
        
        # Deconvolution pars
        'deconvMethod': 'ogb1Kern',
        })


# Build save file name
p['saveFid'] = p['saveFid']+"_"+p['groupVClu']+"_"+p['deconvMethod']
if p['doResample']:
    p['saveFid'] = p['saveFid']+"_"+str(p['newSRate'])+"Hz"+p['fileFmt']
else:
    p['saveFid'] = p['saveFid']+"_"+"7.8Hz"+p['fileFmt']

p

{'groupVClu': 'clu',
 'loadDir': '../../data/2P/original/',
 'loadFid': 'BadenEtAl_RGCs_2016_v1.mat',
 'saveDir': '../../data/2P/proc/',
 'saveFid': 'BadenEtAl_RGCs_2016_ogb1_clu_ogb1Kern_7.8Hz.mat',
 'fileFmt': '.mat',
 'stim': ['chirp', 'bar'],
 'rgcRangeGroup': [1, 32],
 'rgcRangeClu': [1, 49],
 'doNorm': True,
 'normMode': 'medMax',
 'doResample': False,
 'newSRate': 40,
 'deconvMethod': 'ogb1Kern'}

# Load

In [3]:
# Load Baden2016 data
ogb1 = sio.loadmat(p['loadDir']+p['loadFid'])
sio.whosmat(p['loadDir']+p['loadFid'])

[('ans', (1, 1), 'double'),
 ('noise_time', (11210, 1), 'struct'),
 ('noise_trace', (1750, 11210), 'double'),
 ('noise_stim', (20, 15, 1750, 171), 'double'),
 ('cluster_idx', (11210, 1), 'double'),
 ('group_idx', (11210, 1), 'double'),
 ('c2g', (1, 75), 'double'),
 ('sel_idx', (11210, 1), 'logical'),
 ('cell_dsi', (11210, 1), 'double'),
 ('cell_dp', (11210, 1), 'double'),
 ('cell_osi', (11210, 1), 'double'),
 ('cell_op', (11210, 1), 'double'),
 ('cell_area', (11210, 1), 'double'),
 ('cell_volume', (11210, 1), 'double'),
 ('cell_id', (11210, 7), 'double'),
 ('cell_oo_idx', (11210, 1), 'double'),
 ('cell_ff_idx', (11210, 1), 'double'),
 ('chirp_avg', (249, 11210), 'double'),
 ('chirp_byrepeat', (249, 5, 11210), 'double'),
 ('chirp_time', (1, 249), 'double'),
 ('chirp_stim', (31988, 1), 'double'),
 ('chirp_stim_time', (1, 31988), 'double'),
 ('chirp_qi', (11210, 1), 'double'),
 ('chirp_scaling', (11210, 1), 'double'),
 ('bar_byrepeat', (32, 8, 3, 11210), 'double'),
 ('bar_tc', (32, 11210)

## Get Ca++ intensity traces of ROIs stimuli: trial averages

In [4]:
# Chirp stim
chirpAvg = ogb1['chirp_avg']
print(chirpAvg.shape)

# DS stim
barAvg = ogb1['bar_tc']
print(barAvg.shape)

(249, 11210)
(32, 11210)


## Get stimulus times

In [5]:
# Chirp
chirpTime = ogb1['chirp_time'][0]
print(chirpTime.shape)
print('chirpTime:', chirpTime[0], chirpTime[-1])
chirpDur = chirpTime[-1] - chirpTime[0]
print('chirpDur:', chirpDur)
p['chirpFps'] = chirpTime.shape[0] / chirpDur
print('chirpFps:', p['chirpFps'])

chirpStimTime = ogb1['chirp_stim_time'][0]
print(chirpStimTime[-1])

# Bar
barTime = ogb1['bar_time'][0]
print(barTime.shape)
print('barTime:', barTime[0], barTime[-1])
barDur = barTime[-1] - barTime[0]
print('barDur:', barDur)
p['barFps'] = barTime.shape[0] / barDur
print('barFps:', p['barFps'])

(249,)
chirpTime: 0.06415421686746987 31.884645783132527
chirpDur: 31.82049156626506
chirpFps: 7.825146242051799
32.0
(32,)
barTime: 0.0 4.0
barDur: 4.0
barFps: 8.0


## Get group/cluster indices of ROIs

In [6]:
# RGCs: 1-28, uncertain: 29-32, other: > 32

# Get groups or clusters depending on parameter setting
if p['groupVClu'] == 'clu':
    indices = ogb1['cluster_idx']
elif p['groupVClu'] == 'group':
    indices = ogb1['group_idx']

print('nCells:', indices.shape)
print('unique indices:\n', np.unique(indices)) # check that indices contains indices as specified in help text

nCells: (11210, 1)
unique indices:
 [-1  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47
 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71
 72 73 74 75]


## Get quality indices (QI) for cells - see Baden2016 or getGcamp6Data for details

In [7]:
# Chirp
chirpQi = ogb1['chirp_qi']
print(chirpQi.shape)
print(np.min(chirpQi))
print(np.max(chirpQi))
print(np.sort(chirpQi, axis=0)[0:10,0])

# Bar
barQi = ogb1['bar_qi']
print(barQi.shape)
print(barQi[0:10,0])
print(np.min(barQi))
print(np.max(barQi))
print(np.sort(barQi, axis=0)[0:10,0])

(11210, 1)
0.09830159693956375
1.0
[0.0983016 0.120863  0.128501  0.140333  0.153041  0.158132  0.158632
 0.164066  0.164329  0.166061 ]
(11210, 1)
[0.45854601 0.92307502 0.49250901 0.90350097 0.86311299 0.86117202
 0.34725001 0.64195597 0.867962   0.84226602]
0.2651970088481903
0.989113986492157
[0.26519701 0.271816   0.302178   0.323476   0.32687101 0.330311
 0.331785   0.33274299 0.33759701 0.33906299]


## Get cell direction selectivity
cell direction selectivity will be used as step 1 in pre-clustering gcmap6 cells into DS or non-DS

In [8]:
# Get cell Direction selectivity index (DSI)
cellDsi = ogb1['cell_dsi']
print('example cell DSIs:\n', cellDsi[0:10].T)
# Get cell DSI p-value
cellDsiPval = ogb1['cell_dp']
print('examples cell DSI p-vals:\n', cellDsiPval[0:10].T)

# Make logical index of DSI-cells := DSI-Pval <= 0.05
cellDsiBool = np.zeros((len(cellDsi),1), dtype=int)
for idx, val in enumerate(cellDsiPval):
    if val <= 0.05:
        cellDsiBool[idx] = 1

print('examples: DS or not:\n', cellDsiBool[0:10].T)

example cell DSIs:
 [[0.28003699 0.40977299 0.31827399 0.48289701 0.25057    0.36130801
  0.109525   0.34162799 0.336631   0.51661801]]
examples cell DSI p-vals:
 [[0.62599999 0.004      0.16       0.         0.204      0.002
  0.66100001 0.18000001 0.008      0.093     ]]
examples: DS or not:
 [[0 1 0 1 0 1 0 0 1 0]]


## Get cell soma area
Soma size (area) of groups will be used as step 2 (after DS-pre-clustering) in pre-clustering cells into large vs small

In [9]:
cellArea = ogb1['cell_area']
print(cellArea.T)

[[24.22489929 43.06639862 34.99150085 ... 40.3748     64.5996
  51.1414    ]]


# Process data

## Reduce data to RGC groups/clusters only

In [10]:
### Get average traces of RGCs only
if p['groupVClu'] == 'group':
    rgcRange = p['rgcRangeGroup']
elif p['groupVClu'] == 'clu':
    rgcRange = p['rgcRangeClu']

# Chirp
chirpAvgRgc = chirpAvg[:, np.where((indices>=rgcRange[0]) & (indices<=rgcRange[1]))[0]]
print(chirpAvgRgc.shape)

# Bar
barAvgRgc = barAvg[:, np.where((indices>=rgcRange[0]) & (indices<=rgcRange[1]))[0]]
print(barAvgRgc.shape)

(249, 5024)
(32, 5024)


In [11]:
### Get indices of RGCs only
indicesRgc = indices[(indices>=rgcRange[0]) & (indices<=rgcRange[1])]
print(indicesRgc.shape)
print(np.unique(indicesRgc))
print(indicesRgc[0:20]) # print example

(5024,)
[ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24
 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48
 49]
[18 17 44 23 20 22 29 22 22 41 17 20 20 14 31 26 44  9 23 22]


In [12]:
### Get cellDsiBool of RGCs only
cellDsiBool = cellDsiBool[(indices>=rgcRange[0]) & (indices<=rgcRange[1])]
print(cellDsiBool.shape)

(5024,)


## Resample OGB-1 data

In [13]:
if p['doResample']:
    print('Resampling to %.3f Hz' %p['newSRate'])
    # Chirp
    p['chirpFps'] = p['newSRate']
    print(chirpAvgRgc.shape)
    chirpAvgRgc = pop.interpNewSRate(chirpAvgRgc, p['chirpFps'], chirpDur)
    print(chirpAvgRgc.shape)

    # Bar
    p['barFps'] = p['newSRate']
    print(barAvgRgc.shape)
    barAvgRgc = pop.interpNewSRate(barAvgRgc, p['barFps'], barDur)
    print(barAvgRgc.shape)

In [14]:
# Adjust stimulus times to resampled traces
chirpTime = np.linspace(chirpTime[0], chirpTime[-1], chirpAvgRgc.shape[0])
print(chirpTime.shape)

barTime = np.linspace(barTime[0], barTime[-1], barAvgRgc.shape[0])
print(barTime.shape)

(249,)
(32,)


## Get avg traces and deconvolved traces of each RGC group for chirp and bar stimulus
- NOTE: mean traces are normalized like in Baden2016 <> deconvolved traces are range-normalized (due to non-negativity of firing rates) 

In [16]:
# Time cell execution
start = time.time()

## Chirp stimulus
# Initialize 2D arrays
tPoints = chirpAvgRgc.shape[0]
nGroups = np.unique(indicesRgc).shape[0]
chirpAvgRgcMean = np.zeros((tPoints, nGroups))
chirpAvgRgcMed = np.zeros((tPoints, nGroups))
chirpAvgRgcSd = np.zeros((tPoints, nGroups))
chirpAvgRgcDeconvMean = np.zeros((tPoints, nGroups))
chirpAvgRgcDeconvSd = np.zeros((tPoints, nGroups))
chirpN = np.zeros((1, nGroups), dtype=int)

# Process each RGC cluster/group
for idx, iRGC in enumerate(np.unique(indicesRgc)):
    
    print('chirp %s %i' %(p['groupVClu'], iRGC))
    
    # Get mean (normalize for mean)
    chirpAvgRgc[:, indicesRgc==iRGC] = pop.normalize(chirpAvgRgc[:, indicesRgc==iRGC], mode='meanMax')
    chirpAvgRgcMean[:, idx] = np.mean(chirpAvgRgc[:, indicesRgc==iRGC], axis=1)
    chirpAvgRgcSd[:, idx] = np.std(chirpAvgRgc[:, indicesRgc==iRGC], axis=1)

    # Get median (normalize for median)
    chirpAvgRgc[:, indicesRgc==iRGC] = pop.normalize(chirpAvgRgc[:, indicesRgc==iRGC], mode='medMax')
    chirpAvgRgcMed[:, idx] = np.median(chirpAvgRgc[:, indicesRgc==iRGC], axis=1)
    
    chirpN[0, idx] = chirpAvgRgc[:, indicesRgc==iRGC].shape[1]
    
    ## Deconvolve    
    # mean(Deconv single traces): deconvolve single traces, then take the mean
    traces = chirpAvgRgc[:, indicesRgc==iRGC]
    d, traceSm = pop.deconv(traces, fps=p['chirpFps'], method=p['deconvMethod'], smooth=True, norm=True)
    d = pop.normalize(d, mode='meanR') # Normalize 
    chirpAvgRgcDeconvMean[:,idx] = np.mean(d, axis=1)
    chirpAvgRgcDeconvSd[:,idx] = np.std(d, axis=1)

print(chirpAvgRgcMean.shape)

## Bar stimulus
# Initialize 2D array
tPoints = barAvgRgc.shape[0]
nGroups = np.unique(indicesRgc).shape[0]
barAvgRgcMean = np.zeros((tPoints, nGroups))
barAvgRgcMed = np.zeros((tPoints, nGroups))
barAvgRgcSd = np.zeros((tPoints, nGroups))
barAvgRgcDeconvMean = np.zeros((tPoints, nGroups))
barAvgRgcDeconvSd = np.zeros((tPoints, nGroups))
barN = np.zeros((1, nGroups), dtype=int)

# Process each RGC group
for idx,iRGC in enumerate(np.unique(indicesRgc)):
    
    print('bar %s %i' %(p['groupVClu'], iRGC))
    
    # Get mean (normalize for mean)
    barAvgRgc[:, indicesRgc==iRGC] = pop.normalize(barAvgRgc[:, indicesRgc==iRGC], mode='meanMax')
    barAvgRgcMean[:, idx] = np.mean(barAvgRgc[:, indicesRgc==iRGC], axis=1)
    barAvgRgcSd[:, idx] = np.std(barAvgRgc[:, indicesRgc==iRGC], axis=1)

    # Get median (normalize for median)
    barAvgRgc[:, indicesRgc==iRGC] = pop.normalize(barAvgRgc[:, indicesRgc==iRGC], mode='medMax')
    barAvgRgcMed[:, idx] = np.median(barAvgRgc[:, indicesRgc==iRGC], axis=1)
    
    barN[0, idx] = barAvgRgc[:, indicesRgc==iRGC].shape[1]    
    
    ## Deconvolve
    # mean(Deconv single traces)
    traces = barAvgRgc[:, indicesRgc==iRGC]
    d, traceSm = pop.deconv(traces, fps=p['barFps'], method=p['deconvMethod'], smooth=True, norm=True)
    d = pop.normalize(d, mode='meanR') # Normalize 
    barAvgRgcDeconvMean[:,idx] = np.mean(d, axis=1)
    barAvgRgcDeconvSd[:,idx] = np.std(d, axis=1)
    barAvgRgcDeconvMean[:,idx] = pop.normalize(np.mean(d, axis=1), mode='r') # Normalize
    
print(barAvgRgcMean.shape)

print('DONE.')
dur = (time.time() - start)
print('Duration: %.2f sec = %.2f min' %(dur, dur/60))

chirp clu 1
chirp clu 2
chirp clu 3
chirp clu 4
chirp clu 5
chirp clu 6
chirp clu 7
chirp clu 8
chirp clu 9
chirp clu 10
chirp clu 11
chirp clu 12
chirp clu 13
chirp clu 14
chirp clu 15
chirp clu 16
chirp clu 17
chirp clu 18
chirp clu 19
chirp clu 20
chirp clu 21
chirp clu 22
chirp clu 23
chirp clu 24
chirp clu 25
chirp clu 26
chirp clu 27
chirp clu 28
chirp clu 29
chirp clu 30
chirp clu 31
chirp clu 32
chirp clu 33
chirp clu 34
chirp clu 35
chirp clu 36
chirp clu 37
chirp clu 38
chirp clu 39
chirp clu 40
chirp clu 41
chirp clu 42
chirp clu 43
chirp clu 44
chirp clu 45
chirp clu 46
chirp clu 47
chirp clu 48
chirp clu 49
(249, 49)
bar clu 1
bar clu 2
bar clu 3
bar clu 4
bar clu 5
bar clu 6
bar clu 7
bar clu 8
bar clu 9
bar clu 10
bar clu 11
bar clu 12
bar clu 13
bar clu 14
bar clu 15
bar clu 16
bar clu 17
bar clu 18
bar clu 19
bar clu 20
bar clu 21
bar clu 22
bar clu 23
bar clu 24
bar clu 25
bar clu 26
bar clu 27
bar clu 28
bar clu 29
bar clu 30
bar clu 31
bar clu 32
bar clu 33
bar clu 

## save

In [55]:
# Check save directory
print('Saving to:', p['saveDir']+p['saveFid'])

Saving to: ../../data/2P/proc/BadenEtAl_RGCs_2016_ogb1_clu_ogb1Kern_7.8Hz.mat


In [56]:
# Save .mat file
sio.savemat(p['saveDir'],
            {'chirpMean': chirpAvgRgcMean,
             'chirpMed': chirpAvgRgcMed,
             'chirpSd': chirpAvgRgcSd,
             'chirpDeconvMean': chirpAvgRgcDeconvMean,
             'chirpDeconvSd': chirpAvgRgcDeconvSd,
             'chirpTime': chirpTime,
             'chirpN': chirpN,
             
             'barMean': barAvgRgcMean,
             'barMed': barAvgRgcMed,             
             'barSd': barAvgRgcSd,
             'barDeconvMean': barAvgRgcDeconvMean,
             'barDeconvSd': barAvgRgcDeconvSd,
             'barTime': barTime,
             'barN': barN,
             
             'dsIdx': groupDsIdx,
             'somaSizeIdx': somaSizeindices,
            })
print('Saved to:', p['saveDir']+p['saveFid'])

Saved to: ../../data/2P/proc/
